In [17]:
import pandas as pd
import os
import pickle
import numpy as np

In [11]:
csvfile = os.path.join(os.getcwd(), 'data', 'allwells.csv')
df = pd.read_csv(csvfile)

,Lease,Time (Days),Choke Size,Gas Volume (MMscf),Oil Volume (stb),Water Volume (stb),Gas Lift Inj Volume (MMscf),Casing Pressure (psi(a)),Tubing Pressure (psi(a)),Active Pressure (psi(a)),Line Pressure (psi(a)),Pressure Source,Calculated Sandface Pressure (psi(a))
58121,GOOSE,1.0,NaN,0.000,0.0,85.0,NaN,0.000,2915.000,2915.000,1144.101,Tubing Pressure,6138.515940
58122,GOOSE,2.0,NaN,6.988,0.0,440.0,NaN,0.000,2933.000,2933.000,1177.554,Tubing Pressure,4365.013910
58123,GOOSE,3.0,NaN,8.282,0.0,296.0,NaN,0.000,2979.000,2979.000,1165.605,Tubing Pressure,4375.299055
58124,GOOSE,4.0,NaN,8.298,0.0,200.0,NaN,0.000,2933.000,2933.000,1158.871,Tubing Pressure,4244.234491
58125,GOOSE,5.0,NaN,8.156,0.0,125.0,NaN,0.000,2888.000,2888.000,1156.375,Tubing Pressure,4118.618920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60962,GOOSE,2842.0,NaN,0.000,0.0,0.0,NaN,1724.502,-0.755,-0.755,1.723,Tubing Pressure,NaN
60963,GOOSE,2843.0,NaN,0.000,0.0,0.0,NaN,1723.853,-0.111,-0.111,2.343,Tubing Pressure,NaN
60964,GOOSE,2844.0,NaN,0.000,0.0,0.0,NaN,1723.499,-0.398,-0.398,1.099,Tubing Pressure,NaN
60965,GOOSE,2845.0,NaN,0.000,0.0,0.0,NaN,1722.222,-0.555,-0.555,0.633,Tubing Pressure,NaN


In [31]:
outfile = os.path.join(os.getcwd(), 'data', 'bywell.pkl')
if os.path.exists(outfile):
    os.remove(outfile)
with open(outfile, 'ab+') as file:
    for well in np.unique(df['Lease']):
        df_save = df[df['Lease'] == well]
        df_save = df_save.reset_index(drop=True)
        df_save = df_save.rename(
            columns={
                'Gas Volume (MMscf)': 'Avg. Gas Rate (MMscf/d)',
                'Oil Volume (stb)'  : 'Avg. Oil Rate (stb/d)',
                'Water Volume  (stb)': 'Avg. Water Rate (stb/d)',
                })
        time = df_save['Time (Days)'].values
        gas_rate = df_save['Avg. Gas Rate (MMscf/d)'].values
        cum_gas = np.array([0] + [np.sum(gas_rate[:i]) for i in range(1,len(gas_rate))])
        df_save['Cumulative Gas Production (Bscf)'] = cum_gas/1000
        oil_rate = df_save['Avg. Oil Rate (stb/d)'].values
        cum_oil = np.array([0] + [np.sum(oil_rate[:i]) for i in range(1,len(oil_rate))])
        df_save['Cumulative Oil Production (Mbbl)'] = cum_oil/1000
        wat_rate = df_save['Avg. Water Rate (stb/d)'].values
        cum_wat = np.array([0] + [np.sum(wat_rate[:i]) for i in range(1,len(wat_rate))])
        df_save['Cumulative Water Production (Mbbl)'] = cum_wat/1000
        pickle.dump((well, df_save), file)

df_save

,Lease,Time (Days),Choke Size,Avg. Gas Rate (MMscf/d),Avg. Oil Rate (stb/d),Avg. Water Rate (stb/d),Gas Lift Inj Volume (MMscf),Casing Pressure (psi(a)),Tubing Pressure (psi(a)),Active Pressure (psi(a)),Line Pressure (psi(a)),Pressure Source,Calculated Sandface Pressure (psi(a)),Cumulative Gas Production (Bscf),Cumulative Oil Production (Mbbl),Cumulative Water Production (Mbbl)
0,WARBLER,1.0,NaN,6.96350,0.0,616.00,NaN,7552.285,0.000,7552.285,1101.227,Casing Pressure,9698.129159,0.000000,0.0,0.00000
1,WARBLER,2.0,NaN,22.19179,0.0,1787.00,NaN,8349.871,0.000,8349.871,1203.438,Casing Pressure,10550.988278,0.006964,0.0,0.61600
2,WARBLER,3.0,NaN,29.47411,0.0,1859.00,NaN,8463.829,0.000,8463.829,1245.910,Casing Pressure,10601.782756,0.029155,0.0,2.40300
3,WARBLER,4.0,NaN,28.87329,0.0,1449.00,NaN,8416.001,0.000,8416.001,1269.974,Casing Pressure,10434.416628,0.058629,0.0,4.26200
4,WARBLER,5.0,NaN,21.98948,0.0,773.00,NaN,8566.366,0.000,8566.366,1258.577,Casing Pressure,10417.722804,0.087503,0.0,5.71100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,WARBLER,921.0,NaN,3.42783,NaN,92.78,NaN,2003.705,1173.269,1173.269,1108.905,Tubing Pressure,2220.401432,9.221540,NaN,268.06414
921,WARBLER,922.0,NaN,3.03719,NaN,77.96,NaN,2048.913,1291.431,1291.431,1205.878,Tubing Pressure,2246.899504,9.224968,NaN,268.15692
922,WARBLER,923.0,NaN,3.33799,NaN,80.98,NaN,2017.068,1234.002,1234.002,1177.880,Tubing Pressure,2244.014678,9.228005,NaN,268.23488
923,WARBLER,924.0,NaN,3.21693,NaN,75.45,NaN,2021.483,1228.344,1228.344,1211.665,Tubing Pressure,2207.394497,9.231343,NaN,268.31586
